In [1]:
!pip install mysqlclient

In [2]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import requests as r
import calendar
import datetime
import MySQLdb

bd_credentials = {
    'user': 'root',
    'pwd': ''
}

## Functions

In [3]:
##regex to find news on HTML
pattern_news = re.compile('N[1-9]')
##then crawler have to use more than one strategy to get data
pattern_title = re.compile('[1-9] -')

d = {
    'janeiro': '01',
    'fevereiro': '02',
    'março': '03',
    'abril': '04',
    'maio': '05',
    'junho': '06',
    'julho': '07',
    'agosto': '08',
    'setembro': '09',
    'outubro': '10',
    'novembro': '11',
    'dezembro': '12'
}



#fill nan titles
def fill_nan_noticia_title(df):
    for i in range(len(df['noticia-titulo'])):
        if (pd.isna(df['noticia-titulo'][i]) and re.search(pattern_title, df['noticia-strong'][i].__str__())):
            df['noticia-titulo'][i] = df['noticia-strong'][i]

## Split title
def split_title(df):
    for k, v in enumerate(df['noticia-titulo']):
        df['noticia-titulo'][k] = re.split(r'[2-9] -|[2-9]\)', v.__str__() )

def parse_date(df):
    number_list = []
    date_list = []
    
    for k,v in enumerate(df['boletim-titulo']):
        number, date = re.split(r'[\-\|]', v)
        number = re.sub("\D", '', number)
        number_list.append(number)
        try:
            day, month, year = date.split(' de ')
            month = d[month]
        except:
            print(k)
            day, month, year = date.split('/')
        
        day = day.replace('º','')
        date_list.append('{}-{}-{}'.format(year,month,day).replace(' ',''))
        
    df['numero'] = number_list
    df['data'] = date_list


## insert boletim on db
def insert_boletim(df):
    con = MySQLdb.connect('localhost', bd_credentials['user'], bd_credentials['pwd'])
    con.select_db('doug_db')
    cursor = con.cursor()
    for k, v in enumerate(df['news']):
        sqlcommand = 'INSERT INTO  doug_server_app_boletim (data, numero) VALUES ("{}", {});'.format(df['data'][k], int(df['numero'][k]))
        cursor.execute(sqlcommand)
    con.commit()
    con.close()


## Reading CSV File

The csv file contains a amount of html from pages crawled by scrapy

In [4]:
df = pd.read_csv('file.csv', encoding='utf-8')
df['noticia-titulo']

0     <strong>1 - Conep aprova Calendário Universitá...
1     <strong>1) Pedidos de Acesso e Renovação ao Pr...
2     <strong>1 - Programa de Pós-Graduação em Ciênc...
3     <strong>1 - Programação de férias no SIGRH enc...
4     <strong>1- FAUF é credenciada pelo CNPq para r...
5     <strong>1 - Fauf esclarece sobre pagamento das...
6     <strong>1 - Comissão de Acessibilidade da UFSJ...
7     <strong>1 - Proex lança portal do Comunica Ext...
8     <strong>1 - Informativo Dipes/Progp nº 13/2018...
9     <strong>1- Progp abre inscrições para o curso ...
10    <strong>1 - Espetáculo "Cérebro_Coração", com ...
11    <strong>1 - Palestra sobre história do Grupo d...
12    <strong>1 - Inscrições abertas para o II Semin...
13    <strong>1 - UFSJ na luta contra a violência</s...
14    <strong>2- Resolução CGEN Nº19 estabelece alte...
15    <strong>1 - Programa de Pós-Graduação em Educa...
16    <strong><span style="font-size:14px;"><span st...
17    <strong>1 - Fórum discute coleta seletiva 

#### HTML clear

Iterate all html removing all DOM, controller characters and punctuation

In [5]:
for k,v in enumerate(df['noticia-titulo']):
    if(not(pd.isna(v))):
        new = BeautifulSoup(v.__str__(), 'html.parser').text
        new = re.sub('\xa0',' ', new )
        new = re.sub('\n|\t|\r', '', new)
        new = re.sub(',', '', new)
        df['noticia-titulo'][k] = new

df.dropna(subset=['noticia-titulo'], inplace=True)
df.reset_index(drop= True, inplace= True)


## Number of missing values from df new's title

In [6]:
pd.isna(df['noticia-titulo']).sum()

0

## Parsing of titles and dates
Extracting dates and boletins number

In [7]:
split_title(df)
parse_date(df)

16


## Cleaning Html on News body
Removing all DOM and controller characters from new's body

In [8]:
for k, v in enumerate(df.news):
    df.news[k]= BeautifulSoup(v.__str__(), 'html.parser').text

In [9]:
for k, v in enumerate(df.news):
    df.news[k] = re.sub('\n|\t|\r', '',v)
    df.news[k] = re.sub('\xa0', ' ', df.news[k])

## Spliting News according with titles
When news are extracted from UFSJ official site, we need to split news in order to make every independent from anothers, in that way we turn better chatbot's answer to user

In [10]:
news = []
for index, new in enumerate(df.news):
    aux = []
    corte = new
    titulos = df['noticia-titulo'][index]
    titulos.reverse()
    titulos = list(filter(None, titulos))

    for k, v in enumerate(titulos):
        corte = corte.split(v)
        
        try:
            aux.append(corte[1])
            
        except:
            aux.append(corte[0])
            
        corte = corte[0]
        
    titulos.reverse()
    df['noticia-titulo'][index] = titulos
    aux.reverse()
    df.news[index] = aux
print('number of boletins extracted from site: {}'.format(len(df.news)))


number of boletins extracted from site: 83


## Resulted DataFrame


In [11]:
df

,boletim-titulo,noticia-titulo,news,numero,data
0,Boletim nº 9 - 9 de outubro de 2018,[1 - Conep aprova Calendário Universitário de ...,"[O Conselho de Ensino, Pesquisa e Extensão (Co...",9,2018-10-9
1,Boletim nº 12 - 15 de outubro de 2018,[1) Pedidos de Acesso e Renovação ao Protocolo...,[ A Divisão de Prefeitura (Dipre) informa aos ...,12,2018-10-15
2,Boletim nº 10 - 10 de outubro de 2018,[1 - Programa de Pós-Graduação em Ciências Far...,[Boletim nº 10 - 10 de outubro de 20181 - Prog...,10,2018-10-10
3,Boletim nº 11 - 11 de outubro de 2018,[1 - Programação de férias no SIGRH encerra na...,[Boletim nº 11 - 11 de outubro de 20181 - Prog...,11,2018-10-11
4,Boletim nº 14 - 17 de outubro de 2018,[1- FAUF é credenciada pelo CNPq para receber ...,[Boletim nº 14 - 17 de outubro de 20181- FAUF ...,14,2018-10-17
5,Boletim nº 16 - 19 de outubro de 2018,[1 - Fauf esclarece sobre pagamento das bolsas...,[A Fundação de Apoio à Universidade Federal de...,16,2018-10-19
6,Boletim nº 15 | 18 de outubro de 2018,[1 - Comissão de Acessibilidade da UFSJ convid...,[ A Comissão de Acessibilidade da Universidade...,15,2018-10-18
7,Boletim nº 13 - 16 de outubro de 2018,"[1 - Proex lança portal do Comunica Extensão, ...",[ O projeto Comunica Extensão é o canal de div...,13,2018-10-16
8,Boletim nº 19 - 24 de outubro de 2018,"[1 - Informativo Dipes/Progp nº 13/201, Cadas...","[8 -, A Divisão de Administração de Pessoal in...",19,2018-10-24
9,Boletim nº 17 - 22 de outubro de 2018,[1- Progp abre inscrições para o curso Gestão ...,[ A Pró-Reitoria de Gestão e Desenvolvimento d...,17,2018-10-22


# Insert Values into DB
## Insert "Boletins"

In [12]:
insert_boletim(df)

In [13]:
drop = 0
for i, news in enumerate(df.news):
    for j, new in enumerate(news):
        ##find id from boletim
        con = MySQLdb.connect('localhost', bd_credentials['user'], bd_credentials['pwd'])
        con.select_db('doug_db')
        con.set_character_set('utf8')

        cursor = con.cursor()
        print(i)
        cursor.execute('SELECT * from doug_server_app_boletim WHERE numero="{}" '.format(df['numero'][i]))
        instance = cursor.fetchone()
        data_upload = datetime.datetime.now().strftime("%Y-%m-%d")
        disponivel_em = 'https://ufsj.edu.br/ascom/boletim0{}.php'.format(df.numero[k])
        titulo = df['noticia-titulo'][i][j]
        corpo = new
        user_id = None
        boletim_fk_id = instance[0]
        print(titulo)
        sqlInsertString = "INSERT into doug_server_app_noticia(data_upload, disponivel_em, corpo, user_id, boletim_fk_id, titulo) VALUES('{}', '{}', '{}', {}, {}, '{}')".format(
                            data_upload, disponivel_em, corpo, 'NULL', boletim_fk_id, titulo)
        try:
            cursor.execute(sqlInsertString)
        except:
            df.drop(df.index[i])
            drop += 1
        con.commit()
        con.close()

0
1 - Conep aprova Calendário Universitário de 2019
0
 Programa Multicêntrico em Bioquímica e Biologia Molecular lança edital para mestrado e doutorado
0
 CFDD recebe propostas de trabalho sobre interesses difusos e coletivos até o dia 17 de outubroHospital Alemão Oswaldo Cruz e Ministério de Saúde abrem edital para apoio a estudos e tecnologias em saúde
1
1) Pedidos de Acesso e Renovação ao Protocolo/Sipac devem ser encaminhados para o Arquivo Permanente
1
 Netec lança edital para bolsistas de Administração e Comunicação Social (Jornalismo)
2
1 - Programa de Pós-Graduação em Ciências Farmacêuticas lança edital para seleção de mestradoServiço 
2
  Demat convida a comunidade para o Seminário Café com Matemática 
2
 Boletim Informativo: Qualidade de Vida em Foco
3
1 - Programação de férias no SIGRH encerra na próxima segunda-feira (15/10)data limite para que as chefias homologuem as férias de seus servidores e haja tempo hábil para registro no Siapenão programarem até 15/10/2018as férias

 Aquisição de Gases Especiais - Ata de Registro de PreçosDados do Pregão Eletrônico: PE 093/2017
27
 Programas de Pós-Graduação convidam para defesa de tese e exame de qualificaçãoPrograma de Pós-Graduação em Química (exame de qualificação)Programa de Pós-Graduação em Bioengenharia (defesa de tese)
27
 Eleição para a Diretoria Executiva do Sinds UFSJ será amanhã dia 5
28
1 - Eleições para CPPD e Congregação ocorrem nesta quinta-feira (29/11)
28
 Cartas de apresentação dos candidatos à Congregação- José Ricardo Braga- Michel Alves da Silva
28
 Convite para as reuniões do GT Multicampi do PDI
28
 Programas de Pós-Graduação convidam para exames de qaulificação e defesas de dissertação
28
 VIII Prêmio de Fotografia (CNPq) está com inscrições abertas até 18 de janeiro
28
 INFORMATIVO DIPES/PROGP- Nº 14/2018
29
1 - Deceb/Sete Lagoas seleciona servidor para transporte de alunos a São João del-Rei
29
 Chamada pública seleciona projetos para recuperação da bacia do Rio Doce
29
 Processo Seletiv

1 - Pós-Graduação em Geografia convida para defesa de dissertação
73
 Exames de qualificação da Pós-Graduação em Tecnologias para o Desenvolvimento Sustentável 
74
1 - Pedidos para instalação de divisórias balcões bancadas e armários nos campi de São João del-Rei devem ser feitos até 22 de março
75
1 - Progp divulga chamada interna para estágios
75
 Seleção de professor bolsista para o Programa Idiomas sem Fronteiras - Português
75
 UFSJ passa a ser centro aplicador da prova do Celpe-Bras
75
 Pedidos para instalação de divisórias balcões bancadas e armários nos campi de São João del-Rei devem ser feitos até 22 de março
75
 PMBqBM convida para defesa de dissertação
75
 Defesas de dissertação da Pós-Graduação em Engenharia Mecânica
76
1 - Lançado edital para o Programa de Qualificação e Educação Profissional dos Servidores da UFSJ (PQualis)
76
 Progp inscreve para curso de capacitação sobre o Sistema de Concessão de Diárias e Passagens (SCDP)
76
 Crise hídrica em pauta: simpósio internac